# [参考: Python 標準プロファイラ cProfile の簡単な使い方](https://www.qoosky.io/techs/7124adc14d)

In [36]:
%run ../src/cprofile.py

         2 function calls in 1.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.000    1.000    1.000    1.000 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- 上記からわかること
    - 関数が実行された回数は 2 回で 1.000 秒を要した
    - 関数名の順に表示
    - ncalls 各関数の実行回数
    - tottime 各関数の実行に要した時間 (内部でサブ関数の実行に要した時間を除く)
    - percall tottime / ncalls
    - cumtime 各関数の実行に要した時間 (内部でサブ関数の実行に要した時間を含む)
    - percall cumtime / ncalls

In [37]:
%run ../src/cprofile_write.py

In [38]:
!ls ../data/

sample.profile


In [44]:
# NOTE: 読み込み方法がわからない。

# [参考: Python: profile/cProfile モジュールでボトルネックを調べる](https://blog.amedama.jp/entry/2016/08/30/214718)

In [46]:
!sw_vers
!python --version

ProductName:	Mac OS X
ProductVersion:	10.15.7
BuildVersion:	19H1217
Python 3.7.6


In [47]:
import cProfile
import time

## 簡単な例題

In [48]:
cProfile.run('time.sleep(1)')

         4 function calls in 1.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.002    1.002 <string>:1(<module>)
        1    0.000    0.000    1.002    1.002 {built-in method builtins.exec}
        1    1.002    1.002    1.002    1.002 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- ncalls
  - 呼び出し回数
- tottime
  - その関数の実行にかかった時間の合計
  - (別の関数呼び出しにかかった時間を除く)
- percall
  - かかった時間を呼び出し回数で割ったもの
  - (つまり 1 回の呼び出しにかかった平均時間)
- cumtime
  - その関数の実行にかかった時間の合計
  - (別の関数呼び出しにかかった時間を含む)
- filename
  - lineno(function): 関数のファイル名、行番号、関数名

## 再帰的な処理を含む場合

In [100]:
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib(n - 2) + fib(n - 1)

In [101]:
cProfile.run("fib(10)")

         180 function calls (4 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    177/1    0.000    0.000    0.000    0.000 <ipython-input-100-606585d396a7>:1(fib)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- 177/1 といったように分数のような表示が得られる。
- 元々は1回の呼び出しから関数が再帰的に176回呼びだされたことを意味している。

## 実践的な例

### 結果を詳細に見る

In [102]:
#  Profile クラスのインスタンスを用意する。
pr = cProfile.Profile()

# プロファイリングを開始するために enable() メソッドを呼びだそう。
pr.enable()

# その上で、プロファイリングしたいコードを実行する。
fib(10)

# プロファイリングを終了するために disable() メソッドを呼び出す。
pr.disable()

# プロファイリングが終わったら print_stats() メソッドで結果を出力する。
pr.print_stats()

         224 function calls (48 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    177/1    0.000    0.000    0.000    0.000 <ipython-input-100-606585d396a7>:1(fib)
        1    0.000    0.000    0.000    0.000 <ipython-input-102-c311bc68edc5>:11(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-102-c311bc68edc5>:8(<module>)
        2    0.000    0.000    0.000    0.000 codeop.py:135(__call__)
        4    0.000    0.000    0.000    0.000 compilerop.py:138(extra_flags)
        2    0.000    0.000    0.000    0.000 contextlib.py:107(__enter__)
        2    0.000    0.000    0.000    0.000 contextlib.py:116(__exit__)
        2    0.000    0.000    0.000    0.000 contextlib.py:237(helper)
        2    0.000    0.000    0.000    0.000 contextlib.py:81(__init__)
        2    0.000    0.000    0.000    0.000 hooks.py:103(__call__)
        2    0.000    0.000    0.000    0.000 hooks.

In [104]:
import pstats

In [106]:
stats = pstats.Stats(pr)

In [108]:
stats.sort_stats('ncalls')
stats.print_stats()

         224 function calls (48 primitive calls) in 0.000 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    177/1    0.000    0.000    0.000    0.000 <ipython-input-100-606585d396a7>:1(fib)
        4    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.next}
        4    0.000    0.000    0.000    0.000 /Users/atsushi_ozawa/opt/anaconda3/lib/python3.7/site-packages/IPython/core/compilerop.py:138(extra_flags)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 /Users/atsushi_ozawa/opt/anaconda3/lib/python3.7/contextlib.py:81(__init__)
        2    0.000    0.000    0.000    0.000 /Users/atsushi_ozawa/opt/anaconda3/lib/python3.7/contextlib.py:107(__enter__)
        2    0.000    0.000    

### 結果を保存する

In [109]:
pr.dump_stats('../data/fib.profile')

### 結果を読み込む

In [111]:
stats = pstats.Stats('../data/fib.profile')
stats.print_stats()

Fri Aug 20 12:58:30 2021    ../data/fib.profile

         224 function calls (48 primitive calls) in 0.000 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.next}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        2    0.000    0.000    0.000    0.000 /Users/atsushi_ozawa/opt/anaconda3/lib/python3.7/contextlib.py:81(__init__)
        2    0.000    0.000    0.000    0.000 /Users/atsushi_ozawa/opt/anaconda3/lib/python3.7/contextlib.py:107(__enter__)
        2    0.000    0.000    0.000    0.000 /Users/atsushi_ozawa/opt/anaconda3/lib/python3.7/contextlib.py:116(__exit